In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Financials Slide Duplicate\Financials Replacement Function.ipynb"


### Libraries & reading pickle files

In [2]:
themePath = os.getcwd()+"\Theme1.thmx"

In [3]:
client_manuf = ["Danone Group"]
client_brands = ["Actimel",	"Activia", "Danacol", "Danone", "Danone Griego", "Danone Original",	"Danonino",	"Oikos", "Vitalinea", "Yopro", "Alpro"]
 
decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 
prodORitem = "Item"
categories = ["Yogurt"]
sectors = ["Functionals", "Every Day Nutrition", "Plant Based",	"Protein"]
segments = ["Bifidus", "Cholesterol", "Essential", "Greek",	"Immunity",	"Kefir", "Kids", "Light", "Plant Based", "Protein"]
subsegments= ["Essential Drink", "Essential Spoon",	"Greek Spoon", "Kids Drink", "Kids Spoon", "Light Drink", "Light Spoon"]
subcategories= []

national = True
customareas= ""
areas = ["NATIONAL","RETAILER"]

regions_RET  = ["Carrefour", "Ahorramas"]
channels_RET = ["Carrefour Hyper", "Carrefour Super"]
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | December 2024"

end_date = "2025-01-01"

OpenEditData=True
ManufOrTopC ="Top Companies"
BrandOrTopB="Top Brands"

percent = 100000
percentstr="'00 000"
### OpenEditData is a parameter (run open excel cell or not )
OpenEditData=True


In [4]:
national = ['National'] if "NATIONAL" in areas else []
retailers = regions_RET + channels_RET + market_RET
channels = regions_CHAN + channels_CHAN + market_CHAN
cust = regions_CUST + channels_CUST + market_CUST
area=national+retailers+channels+cust

In [5]:
loaded_data = {}

datasets_path = os.getcwd()+"\\""Financials Datasets""\\"
print(datasets_path)

datasets = os.listdir(datasets_path)
for dataset in datasets:
    file_path = os.path.join(datasets_path, dataset)
    with open(file_path, 'rb') as handle:
        globals()[dataset.split('.')[0]] = pd.read_pickle(handle)

c:\Users\aleaa\Documents\Slide-Automate\Financials Slide Duplicate\Financials Datasets\


### Replacement functions

# Slide1: Mix Analysis

### By Brand

In [6]:
modified_brand_P12M_dfs={}
for k in brands_client_dfs.keys():
    t=brands_client_dfs[k].copy()
    t=DetectHeader(t)
    t=t[:-1]
    t['Value Sales IYA'] = t['Value Sales IYA'].astype(float).fillna(-199)
    t=t.fillna(0)
    t = t[t['Net Sales'].astype(float) >= 1000]
    total= t[(t['Top Brands'].str.contains( ' Total')) & ~(t['Top Brands'].isin(['Grand Total','All Others Total'])) & ~(t['Top Brands'].isin([i+' Total' for i in client_brands]))]
    if not t.empty and len(t['Top Brands']) != 1:
        modified_brand_P12M_dfs[k]=t

### By Market 

In [7]:
def process_RET_CHAN_POS(dataframes,regions, channels, markets):
    modified_regions = {}
    modified_channels = {}
    modified_markets = {}
    dya = {}

    for k in dataframes.keys():
        t = dataframes[k].copy()
        # t.columns = t.iloc[cut_df-1]
        # t = t.iloc[cut_df:]
        t=DetectHeader(t)

        t = t[t["Net Sales"].astype(float)>=1000]
        t['Value Sales IYA'] =t['Value Sales IYA'].astype(float).fillna(-199) 

        level_check = set(["Region", "Channel", "Market"])
        existant_cols = list(set(t.columns)&level_check)

        decider_dic = {"Region": regions, "Channel":channels, "Market":markets}
        grand_tot = t[t[t.columns[0]] == 'Grand Total'] 
        dya[k] = grand_tot

        if len(existant_cols) == 1:
            df = t[t[t.columns[0]].isin(decider_dic[existant_cols[0]])]
            if not df.empty:
                if df.columns[0] == "Region":
                    modified_regions[k] = df
                if df.columns[0] == "Channel":
                    modified_channels[k] = df
                if df.columns[0] == "Market":
                    modified_markets[k] = df

        elif len(existant_cols) == 2:
            levels_rank = {"Region":1, "Channel":2, "Market":3}
            existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])
            t_child = t[t[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns = [existant_cols[0]])
            t_parent = t[t[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns = [existant_cols[1]])
            t_parent[existant_cols[0]] = t_parent[existant_cols[0]].str.replace(" Total", "").str.strip()
            for df in [t_child, t_parent]:
                if not df.empty:
                    if df.columns[0] == "Region":
                        modified_regions[k] = df
                    if df.columns[0] == "Channel":
                        modified_channels[k] = df
                    if df.columns[0] == "Market":
                        modified_markets[k] = df
            

        else:
            levels_rank = {"Region":1, "Channel":2, "Market":3}
            existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])
            t_market = t[t[existant_cols[2]].isin(decider_dic[existant_cols[2]])].drop(columns = [existant_cols[0],existant_cols[1]])
            t_channel = t[t[existant_cols[1]].isin([i + " Total" for i in decider_dic[existant_cols[1]]])].drop(columns = [existant_cols[0],existant_cols[2]])
            t_channel[existant_cols[1]] = t_channel[existant_cols[1]].str.replace(" Total", "").str.strip()
            t_region = t[t[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns = [existant_cols[1],existant_cols[2]])
            t_region[existant_cols[0]] = t_region[existant_cols[0]].str.replace(" Total", "").str.strip()
            if not t_region.empty:
                modified_regions[k] = t_region
            if not t_channel.empty:
                modified_channels[k] = t_channel
            if not t_market.empty:
                modified_markets[k] = t_market
            
    return modified_regions, modified_channels, modified_markets, dya
            



In [8]:
modified_retailer_regions = {}
modified_retailer_channels = {}
modified_retailer_markets = {}

modified_channels_regions = {}
modified_channels_channels = {}
modified_channels_markets = {}

modified_cust_regions = {}
modified_cust_channels = {}
modified_cust_markets = {}

dya_retailer={}
dya_channel={}
dya_cust = {}

#**********Retailer*********
if len(retailers)!=0:
    modified_retailer_regions, modified_retailer_channels, modified_retailer_markets, dya_retailer = process_RET_CHAN_POS(retailers_P12M_dfs, regions_RET, channels_RET, market_RET)
# *********Channels**********
if len(channels)!=0:
    modified_channels_regions, modified_channels_channels, modified_channels_markets, dya_channel = process_RET_CHAN_POS(channel_P12M_dfs, regions_CHAN, channels_CHAN, market_CHAN)
#********POS****************
if len(cust)!=0: 
    modified_cust_regions, modified_cust_channels, modified_cust_markets, dya_cust = process_RET_CHAN_POS(cust_P12M_dfs, regions_CUST, channels_CUST, market_CUST)


### By Sector/Segment

In [9]:
def MixAnalysisCleaning(inputdic):
    modifieddic={}
    for k in inputdic.keys():
        t=inputdic[k].copy()
        t=DetectHeader(t)
        t=t[:-1]
        t['Value Sales IYA'] =t['Value Sales IYA'].astype(float).fillna(-199)
        t=t.fillna(0)
        t=t[t["Net Sales"]>=1000]
        if not t.empty:
            modifieddic[k]=t
    return modifieddic   


In [10]:
if len(sectors)!=0:
    modified_sectors_P12M_mix_analysis_dfs=MixAnalysisCleaning(Sector_P12M_client_dfs)
if len(segments)!=0:
    modified_segment_P12M_mix_analysis_dfs=MixAnalysisCleaning(Segment_P12M_client_dfs)
if len(subsegments)!=0:
    modified_subsegment_P12M_mix_analysis_dfs=MixAnalysisCleaning(SubSegment_P12M_client_dfs)
if len(subcategories)!=0:
    modified_subcategory_P12M_mix_analysis_dfs=MixAnalysisCleaning(SubCategory_P12M_client_dfs)

# Slide2: Trade Margin

In [11]:
def TradeMarginCleaning(inputdic,scope=""):
    modifieddic={}
    for k in inputdic.keys():
        t = inputdic[k].copy()
        t=DetectHeader(t)
        t[scope] = t[scope].replace('Grand Total', 'Total')
        t=t.replace(np.nan,0)
        if not t.empty:
            modifieddic[k]=t
    return modifieddic   


In [12]:
if len(sectors)!=0:
    modified_sector_Trade_Margin_dfs=TradeMarginCleaning(Sector_P12M_client_dfs,scope="Sector")
if len(segments)!=0:
    modified_segment_Trade_Margin_dfs=TradeMarginCleaning(Segment_P12M_client_dfs,scope="Segment")

if len(subsegments)!=0:
    modified_subsegment_Trade_Margin_dfs=TradeMarginCleaning(SubSegment_P12M_client_dfs,scope="SubSegment")

if len(subcategories)!=0:
    modified_subcategory_Trade_Margin_dfs=TradeMarginCleaning(SubCategory_P12M_client_dfs,scope="SubCategory")


# Sector KPI

#### Cleaning

In [51]:
def clean(data,lis, Scope ="Sector"):
    final ={}
    for key, df in data.items():
        df=DetectHeader(df)
        df.fillna(0,inplace = True)
        df = df.reset_index(drop=True)
        df =df[df['Net Sales'] > 1000]
        df=df[df[Scope].isin(lis) ]

        df = df[~df[Scope].str.contains('Grand Total', case=False)]
        if df.shape[0] !=0:
            if 'National' in key:
                new_key = key.split(' | ')[0] + ' | ' + key.split(' | ')[2] +' | ' + key.split(' | ')[1]
            else:
                new_key = key
            final[new_key] = df
    return final

In [54]:
def merge_data(data1,data2, Scope = "Sector"):
    final = {}
    for key in modified_sector_P12M.keys():
        if key in modified_sector_P3M.keys():
            final[key] = pd.merge(data1[key], data2[key], on = Scope, suffixes= ("_P12M", "_P3M"))
            df = final[key]
            df = df[~df[Scope].str.contains('Total', case=False)]
            df =df[df['Net Sales_P3M'] > 1000]
            # df=df[df[Scope].isin(lis) ]
            df = df.sort_values('Rate of Sales_P3M', ascending=False).reset_index(drop =True)
            
            
            if df.shape[0]>0:
                final[key] = df
            
    return final

In [55]:
modified_sector_P12M = {}
modified_sector_P3M ={}
merged_data_sector ={}
if len(sectors)!=0:
    modified_sector_P12M  = clean(Sector_P12M_client_dfs,sectors, "Sector")
    modified_sector_P3M  = clean(Sector_P3M_client_dfs,sectors, "Sector")
    merged_data_sector = merge_data(modified_sector_P12M,modified_sector_P3M, "Sector")


modified_segment_P12M ={}
modified_segment_P3M ={}
merged_data_segment ={}
if len(segments)!=0:
    modified_segment_P12M  = clean(Segment_P12M_client_dfs,segments, "Segment")
    modified_segment_P3M  = clean(Segment_P3M_client_dfs,segments, "Segment")
    merged_data_segment = merge_data(modified_segment_P12M,modified_segment_P3M, "Segment")

modified_subsegment_P12M ={}
modified_subsegment_P3M ={}
merged_data_subsegment ={}
if len(subsegments)!=0:
    modified_subsegment_P12M  = clean(SubSegment_P12M_client_dfs,subsegments, "SubSegment")
    modified_subsegment_P3M  = clean(SubSegment_P3M_client_dfs,subsegments, "SubSegment")
    merged_data_subsegment = merge_data(modified_subsegment_P12M,modified_subsegment_P3M, "SubSegment")

modified_subcategory_P12M = {}
modified_subcategory_P3M ={}
merged_data_subcategory ={}
if len(subcategories)!=0:
    modified_subcategory_P12M  = clean(SubCategory_P12M_client_dfs,subcategories, "SubCategory")
    modified_subcategory_P3M  = clean(SubCategory_P3M_client_dfs,subcategories, "SubCategory")
    merged_data_subcategory = merge_data(modified_subcategory_P12M,modified_subcategory_P3M, "SubCategory")


# Slide 4: SKU KPIs Summary 

### By_Product

In [57]:
modified_product_P12M_dfs={}
modified_product_P3M_dfs={}
modified_product_dfs = {}

#P3M
for k in product_P3M_dfs.keys():
    t=product_P3M_dfs[k].copy()
    t=DetectHeader(t)
    t=t[:-1]
    t['Top Brands']=t['Top Brands'].ffill()
    t = t[~t["Top Brands"].str.contains(' Total')]
    df = t[t['Top Brands'].isin(client_brands)]
    df = df.fillna(0)
    df = df[df['Net Sales'] > 1000]
    if not df.empty:
        modified_product_P3M_dfs[k] = df

#P12M
for k in product_P12M_dfs.keys():
    t=product_P12M_dfs[k].copy()
    t=DetectHeader(t)
    t=t[:-1]
    t['Top Brands']=t['Top Brands'].ffill()
    t = t[~t["Top Brands"].str.contains(' Total')]
    df = t[t['Top Brands'].isin(client_brands)]
    df = df.fillna(0)
    df['Net Sales'] = df['Net Sales'].astype(float)
    df = df[df['Net Sales'] > 1000]
    # df = df.nlargest(15, 'Net Sales')
    if not df.empty:
        modified_product_P12M_dfs[k] = df


missing_keys = [k for k in modified_product_P12M_dfs.keys() if k not in modified_product_P3M_dfs]
for k in missing_keys:
    modified_product_P3M_dfs[k] = pd.DataFrame(columns=modified_product_P12M_dfs[k].columns)

for k in modified_product_P12M_dfs.keys():
    p12m_df = modified_product_P12M_dfs[k].copy()
    p12m_df = p12m_df.nlargest(15, 'Net Sales')
    p3m_df = modified_product_P3M_dfs[k].copy()
    comb = pd.merge(p12m_df,p3m_df, how = 'left',on = ["Top Brands",f'{prodORitem}'], suffixes = ("_P12M", "_P3M"))
    comb = comb.sort_values(by=f'{prodORitem} Sales Rate_P3M', ascending=False)
    # Compute Sales Rate Ix
    first_value = comb[f'{prodORitem} Sales Rate_P3M'].iloc[0]
    comb['Sales Rate Ix'] = (comb[f'{prodORitem} Sales Rate_P3M'] / first_value)
    modified_product_dfs[k] = comb.fillna(0)

### By_Brand 


In [58]:
modified_brand_product_P3M_dfs={}
modified_brand_product_P12M_dfs={}
modified_brand_product_dfs={}

###P3M
for k in modified_product_P3M_dfs.keys():
        t = modified_product_P3M_dfs[k].copy()
        for brand in client_brands:
            df = t[t['Top Brands'] == brand]
            new_key = f"{brand} | {k}"
            parts = [part.strip() for part in new_key.split('|') if f'{client_manuf[0]}' not in part]
            new_key = ' | '.join(parts)
            if not df.empty :
                modified_brand_product_P3M_dfs[new_key] = df  


###P12M    
for k in modified_product_P12M_dfs.keys():
        t = modified_product_P12M_dfs[k].copy()
        for brand in client_brands:
            df = t[t['Top Brands'] == brand]
            new_key = f"{brand} | {k}"
            parts = [part.strip() for part in new_key.split('|') if f'{client_manuf[0]}' not in part]
            new_key = ' | '.join(parts)
            if not df.empty :
                modified_brand_product_P12M_dfs[new_key] = df    


missing_keys = [k for k in modified_brand_product_P12M_dfs.keys() if k not in modified_brand_product_P3M_dfs]
for k in missing_keys:
    modified_brand_product_P3M_dfs[k] = pd.DataFrame(columns=modified_brand_product_P12M_dfs[k].columns)

for k in modified_brand_product_P12M_dfs.keys():
    p12m_df = modified_brand_product_P12M_dfs[k].copy()
    p12m_df = p12m_df.nlargest(15, 'Net Sales')
    p3m_df = modified_brand_product_P3M_dfs[k].copy()
    comb = pd.merge(p12m_df,p3m_df, how = 'left',on = ["Top Brands",f'{prodORitem}'], suffixes = ("_P12M", "_P3M"))
    
    comb = comb.sort_values(by=f'{prodORitem} Sales Rate_P3M', ascending=False)
    # Compute Sales Rate Ix
    first_value = comb[f'{prodORitem} Sales Rate_P3M'].iloc[0]
    comb['Sales Rate Ix'] = (comb[f'{prodORitem} Sales Rate_P3M'] / first_value)
    modified_brand_product_dfs[k] = comb.fillna(0)          

# Slide 5_Mix Matrix

In [59]:
def mixMatrixClean(data):
    final ={}
    for key, val in data.items():
        df=val.copy()
        columnsNum = df.loc[df['Unnamed: 1']=='Values'].index[0]+1
        df.columns = df.iloc[columnsNum]
        df = df.iloc[columnsNum+1:].reset_index(drop=True)
        if df.shape[0] !=0:
            if 'National' in key:
                new_key = ' | '.join([key.split(' | ')[0],key.split(' | ')[2],key.split(' | ')[1]])
            else:
                new_key = key

            df.fillna(0,inplace = True)
            df['Source'] = new_key.split(' | ')[2]

            final[new_key] = df.sort_values(by='Value Sales',ascending=False)
    #         final[new_key] = df.sort_values(by='Value Sales',ascending=False)
    sortOrder=final[new_key][final[new_key].columns[0]].unique()

    # print(sortOrder,final[new_key].columns[0])
    for key,val in final.items():
        val['order']=val[val.columns[0]].replace(dict(zip(sortOrder,range(len(sortOrder)))))
        final[key]=val



    return final


In [60]:
if len(sectors)!=0:
    sector_P12M_modified = mixMatrixClean(Sector_P12M_client_dfs)
if len(segments)!=0:
    segment_P12M_modified = mixMatrixClean(Segment_P12M_client_dfs)
if len(subcategories)!=0:
    subCat_P12M_modified = mixMatrixClean(SubCategory_P12M_client_dfs)
if len(subsegments)!=0:
    subSeg_P12M_modified = mixMatrixClean(SubSegment_P12M_client_dfs)
    
    
secRetailerDf,segRetailerDf,subCatRetailerDf,subSegRetailerDf,brandsRetailerDf={},{},{},{},{}
secChannelDf,segChannelDf,subCatChannelDf,subSegChannelDf,brandsChannelDf={},{},{},{},{}
secCustDf,segCustDf,subCatCustDf,subSegCustDf,brandsCustDf={},{},{},{},{}

for key,retailerList in {'region':['National'] if national else []+regions_RET,'channel':['National'] if national else []+channels_RET,'market':['National'] if national else []+market_RET}.items():
    if len(sectors)!=0:secRetailerDf[key] = {key: sector_P12M_modified[key] for key in sector_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}
    if len(segments)!=0:segRetailerDf[key] = {key: segment_P12M_modified[key] for key in segment_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}
    if len(subcategories)!=0:subCatRetailerDf[key] = {key: subCat_P12M_modified[key] for key in subCat_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}
    if len(subsegments)!=0:subSegRetailerDf[key] = {key: subSeg_P12M_modified[key] for key in subSeg_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}

for key,channelList in {'region':['National'] if national else []+regions_CHAN,'channel':['National'] if national else []+channels_CHAN,'market':['National'] if national else []+market_CHAN}.items():
    if len(sectors)!=0:secChannelDf[key] = {key: sector_P12M_modified[key] for key in sector_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}
    if len(segments)!=0:segChannelDf[key] = {key: segment_P12M_modified[key] for key in segment_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}
    if len(subcategories)!=0:subCatChannelDf[key] = {key: subCat_P12M_modified[key] for key in subCat_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}
    if len(subsegments)!=0:subSegChannelDf[key] = {key: subSeg_P12M_modified[key] for key in subSeg_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}

for key,custList in {'region':['National'] if national else []+regions_CUST,'channel':['National'] if national else []+channels_CUST,'market':['National'] if national else []+market_CUST}.items():
    if len(sectors)!=0:secCustDf[key] = {key: sector_P12M_modified[key] for key in sector_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}
    if len(segments)!=0:segCustDf[key] = {key: segment_P12M_modified[key] for key in segment_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}
    if len(subcategories)!=0:subCatCustDf[key] = {key: subCat_P12M_modified[key] for key in subCat_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}
    if len(subsegments)!=0:subSegCustDf[key] = {key: subSeg_P12M_modified[key] for key in subSeg_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}


In [61]:
secRetailerClient,segRetailerClient,subCatRetailerClient,subSegRetailerClient={},{},{},{}
secChannelClient,segChannelClient,subCatChannelClient,subSegChannelClient={},{},{},{}
secCustClient,segCustClient,subCatCustClient,subSegCustClient={},{},{},{}


def clientAgg(dic):

    clients={}
    for marketLevel in ['region','channel','market']:
        for manuf in client_manuf:
            keys = [dic[marketLevel][key] for key in dic[marketLevel].keys() if manuf in key.split(' | ')]
            if keys:
                clients[manuf+' | '+marketLevel]=pd.concat(keys).reset_index(drop = True)
    return clients

if len(sectors)!=0:secRetailerClient = clientAgg(secRetailerDf)
if len(segments)!=0:segRetailerClient = clientAgg(segRetailerDf)
if len(subcategories)!=0:subCatRetailerClient = clientAgg(subCatRetailerDf)
if len(subsegments)!=0:subSegRetailerClient = clientAgg(subSegRetailerDf)

if len(sectors)!=0:secChannelClient = clientAgg(secChannelDf)
if len(segments)!=0:segChannelClient = clientAgg(segChannelDf)
if len(subcategories)!=0:subCatChannelClient = clientAgg(subCatChannelDf)
if len(subsegments)!=0:subSegChannelClient = clientAgg(subSegChannelDf)

if len(sectors)!=0:secCustClient = clientAgg(secCustDf)
if len(segments)!=0:segCustClient = clientAgg(segCustDf)
if len(subcategories)!=0:subCatCustClient = clientAgg(subCatCustDf)
if len(subsegments)!=0:subSegCustClient = clientAgg(subSegCustDf)


In [62]:
retailerDuplication = sum([len(secRetailerClient)+len(segRetailerClient)+len(subCatRetailerClient)+len(subSegRetailerClient)])
channelDuplication = sum([len(secChannelClient)+len(segChannelClient)+len(subCatChannelClient)+len(subSegChannelClient)])
custDuplication = sum([len(secCustClient)+len(segCustClient)+len(subCatCustClient)+len(subSegCustClient)])
retailerDuplication,channelDuplication,custDuplication

(0, 0, 0)

In [63]:
def fillMissingValue(dfToFill):
    for key,val in dfToFill.items():
        colToFill = val.columns[0]
        uniqueValue = val[~val[colToFill].str.contains('Total')][colToFill].unique()
        dfLis = []
        for source in val['Source'].unique():
            df = val[val.Source==source]
            missingValue = list(set(uniqueValue) - set(df[~df[colToFill].str.contains('Total')][colToFill].unique()))
            missingValue = pd.DataFrame({colToFill: missingValue,'Source':source})
            val = pd.concat([val, missingValue]).replace(np.nan,0).reset_index(drop=True)
            
        ## Value Identifier on the avg or national value for the color schema on replacement
        if 'National' in val['Source'].unique():
            val['Net Total'] = val[(val.Source=='National')&(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].iloc[0]
            val['GM Total'] = val[(val.Source=='National')&(val[colToFill].str.contains('Total'))]['Gross Margin %'].iloc[0]
        else:
            val['Net Total'] = val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].sum()/val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].count()
            val['GM Total'] = val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].sum()/val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].count()
        dfToFill[key]=val
        
    return dfToFill
                            
retailerDic = {'Sec':secRetailerClient,'Seg':segRetailerClient,'SubSeg':subSegRetailerClient,'SubCat':subCatRetailerClient}
channelDic = {'Sec':secChannelClient,'Seg':segChannelClient,'SubSeg':subSegChannelClient,'SubCat':subCatChannelClient}
custDic = {'Sec':secCustClient,'Seg':segCustClient,'SubSeg':subSegCustClient,'SubCat':subCatCustClient}

for key,val in retailerDic.items():
    retailerDic[key] = fillMissingValue(val)
for key,val in channelDic.items():
    channelDic[key] = fillMissingValue(val)
for key,val in custDic.items():
    custDic[key] = fillMissingValue(val)

# SLide6: MixMatrix ByBrand

In [64]:
def mixMatrixbybrandClean(data):
    final = {}
    for key, val in data.items():
        df = val.copy()
        df = DetectHeader(df)
       
        if df.shape[0] != 0:
            new_key = key
           
            df.fillna(0, inplace=True)
            if categories[0] in key.split(' | ')[0]:
                df['Source'] = new_key.split(' | ')[0]
            else:
                df['Source'] = new_key.split(' | ')[2]
            # Sort the DataFrame by 'Value Sales'
            final[new_key] = df.sort_values(by='Value Sales', ascending=False)
            # Generate sort order based on sorted unique values of the first column
            sortOrder = final[new_key][final[new_key].columns[0]].unique()
            # Apply the sort order to the current DataFrame
            final[new_key]['order'] = final[new_key][final[new_key].columns[0]].replace(dict(zip(sortOrder, range(len(sortOrder)))))
 
    return final
 

In [65]:
def fillMissingValue(dfToFill):
    for key,val in dfToFill.items():
        colToFill = val.columns[0]
        uniqueValue = val[~val[colToFill].str.contains('Total')][colToFill].unique()
        dfLis = []
        for source in val['Source'].unique():
            df = val[val.Source==source]
            missingValue = list(set(uniqueValue) - set(df[~df[colToFill].str.contains('Total')][colToFill].unique()))
            missingValue = pd.DataFrame({colToFill: missingValue,'Source':source})
            val = pd.concat([val, missingValue]).replace(np.nan,0).reset_index(drop=True)
            
        ## Value Identifier on the avg or national value for the color schema on replacement
            val['Net Total'] = val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].sum()/val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].count()
            val['GM Total'] = val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].sum()/val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].count()
        dfToFill[key]=val
        
    return dfToFill
            

In [66]:
# cat,sec,seg,subseg,subcat 
secbrand,segbrand,subsegbrand,subcatbrand = {},{},{},{}
for key in brands_client_dfs.keys():
    key_split = key.split(' | ')
    if len(sectors) != 0 and len(key_split) > 2 and key_split[2] in sectors or key_split[0] in categories  :secbrand[key] = brands_client_dfs[key]
    if len(segments) != 0 and len(key_split) > 2 and key_split[2] in segments or key_split[0] in categories:segbrand[key] = brands_client_dfs[key]
    if len(subsegments) != 0 and len(key_split) > 2 and key_split[2] in subsegments or key_split[0] in categories:subsegbrand[key] = brands_client_dfs[key]
    if len(subcategories) != 0 and len(key_split) > 2 and key_split[2] in subcategories or key_split[0] in categories:subcatbrand[key] = brands_client_dfs[key]
 
if len(sectors) != 0:
        sec_dfclean=mixMatrixbybrandClean(secbrand)
        sectorbybrand=fillMissingValue(sec_dfclean)
if len(segments) != 0:
        seg_dfclean=mixMatrixbybrandClean(segbrand)
        segmentbybrand=fillMissingValue(seg_dfclean)

if len(subsegments) != 0: 
        subseg_dfclean=mixMatrixbybrandClean(subsegbrand)
        subsegmentbybrand=fillMissingValue(subseg_dfclean)

if len(subcategories) != 0:
       subcat_dfclean=mixMatrixbybrandClean(subcatbrand)
       subcategorybybrand=fillMissingValue(subcat_dfclean)


In [67]:
def concatdf(dic,area):
    final = {}

    for key_part in area:
        print(key_part)
        aligned_dfs = []

        # Filter and align DataFrames for the current area
        for key, df in dic.items():
            key_parts = key.split(" | ")  # Split key into parts based on " | "
            # print(key_parts)
            if key_part in key_parts:  # Check if the key includes the area (e.g., "NATIONAL")
                # print(key)
                # Set 'Top Brands' as index for alignment, reindex to ensure all brands are present
                area_brands = pd.concat([df['Top Brands'] for key, df in dic.items() if key_part in key]).unique()
                aligned_df = df.set_index('Top Brands').reindex(area_brands).reset_index()
                aligned_df['Source'] = aligned_df['Source'].ffill()  # Track the source of the data
                aligned_df = aligned_df.fillna(0)  # Fill missing values with 0
                aligned_df.rename(columns={"Top Brands": client_manuf[0]}, inplace=True)
                aligned_dfs.append(aligned_df)

        if aligned_dfs:  # Ensure there's at least one DataFrame to concatenate
            final_df = pd.concat(aligned_dfs, ignore_index=True)
            final[key_part] = final_df  # Store the concatenated DataFrame in the final dictionary

    return final

In [68]:
if len(sectors)!=0:
    secfinaldic=concatdf(sectorbybrand,area)
if len(segments)!=0:
    segfinaldic=concatdf(segmentbybrand,area)
if len(subsegments)!=0:
    subsegfinaldic=concatdf(subsegmentbybrand,area)
if len(subcategories)!=0:
    subcatfinaldic=concatdf(subcategorybybrand,area)

National
Carrefour
Ahorramas
Carrefour Hyper
Carrefour Super
National
Carrefour
Ahorramas
Carrefour Hyper
Carrefour Super
National
Carrefour
Ahorramas
Carrefour Hyper
Carrefour Super


# Slide7:Sector Spending Pool

### Product Spending Pool

#### Cleaning

In [69]:
def cleanProducts(data):
    final ={}
    for key, df in data.items():
        df=DetectHeader(df)
        df['Top Brands'] = df['Top Brands'].replace(0, np.nan)
        df['Top Brands'] = df['Top Brands'].fillna(method = 'ffill')
        df = df[~df['Top Brands'].str.contains('Total', case=False)]
        df.fillna(0,inplace = True)
        df = df[df['Net Sales']>1000]
        # df = df[df['Top Brands'].isin(client_brands)]
        df = df.sort_values(by= 'Net Sales', ascending=False)
        df =df.head(12)
        df = df.reset_index(drop=True)
        if df.shape[0] !=0:
            if 'National' in key:
                new_key = key.split(' | ')[0] + ' | ' + key.split(' | ')[2] +' | ' + key.split(' | ')[1]
            else:
                new_key = key
            final[new_key] = df
    return final

In [70]:
modified_products_p12m = cleanProducts(product_P12M_dfs)

In [71]:
modified_products_p12m_brand ={}
for key, df in modified_brand_product_P12M_dfs.items():
    df['Top Brands'] = df['Top Brands'].replace(0, np.nan)
    df['Top Brands'] = df['Top Brands'].fillna(method = 'ffill')
    df = df[~df['Top Brands'].str.contains('Total', case=False)]
    df.fillna(0,inplace = True)
    df = df[df['Net Sales']>1000]
    df = df[df['Top Brands'].isin(client_brands)]
    df = df.sort_values(by= 'Net Sales', ascending=False)
    df =df.head(12)
    df = df.reset_index(drop=True)
    if df.shape[0] !=0:
        if 'National' in key:
            new_key = key.split(' | ')[0] + ' | ' + key.split(' | ')[2] +' | ' + key.split(' | ')[1]
        else:
            new_key = key
        modified_products_p12m_brand[new_key] = df



# SKU Profitability Slide 8

### By Product

In [72]:
###P12M
modified_product_sec_seg_P12M_dfs = {}

for k in product_P12M_dfs.keys():
    t=product_P12M_dfs[k].copy()
    t=DetectHeader(t)
    t=t[:-1]
    t['Value Sales IYA'] =t['Value Sales IYA'].astype(float).fillna(-199)
    t['Net Sales'] =t['Net Sales'].astype(float).fillna(0)
    t['Net Sales/Kg'] =t['Net Sales/Kg'].astype(float).fillna(0)

    t['Gross Margin %'] =t['Gross Margin %'].astype(float).fillna(0)

    t['Top Brands']=t['Top Brands'].ffill()
    total= t[(t['Top Brands'].str.contains( ' Total')) & ~(t['Top Brands'].isin(['Grand Total','All Others Total'])) & ~(t['Top Brands'].isin([i+' Total' for i in client_brands]))]
    df = t[t['Top Brands'].isin(client_brands)]
    df[f'{prodORitem} Sales Rate'] = df[f'{prodORitem} Sales Rate'].fillna(0)
    df = df[df['Net Sales'] >= 1000]
    if (not df.empty):
        modified_product_sec_seg_P12M_dfs[k] = df


# Slide9: Trade margin table vs Competition 

### By client and competitor 

In [73]:
def sort_df_excluding_total(df, Inscope=""):
    df = df.fillna(0)
    total_row = df[df[f'{ManufOrTopC}'] == 'Total']
    df = df[df[f'{ManufOrTopC}'] != 'Total']
    df = df.sort_values(by=[Inscope], ascending=[True]).reset_index(drop=True)
    df = pd.concat([df, total_row]).reset_index(drop=True)

    return df

def split_client_copetitor(dictionary,client_dictionary,competitor_dictionary,Inscope=""):
    for k in dictionary.keys():
        t = dictionary[k].copy()
        t=DetectHeader(t)
        t=t[:-1]
        t[f'{ManufOrTopC}'] = t[f'{ManufOrTopC}'].ffill()
        t[Inscope]=t[Inscope].fillna(t[f'{ManufOrTopC}'])
        t['Net Sales'] = t['Net Sales'].astype(float)
        total_entries = t[(t[f'{ManufOrTopC}'].str.contains(' Total')) & ~(t[f'{ManufOrTopC}'].isin(['Grand Total',"All Others Total"])) & ~(t[f'{ManufOrTopC}'].isin([i+' Total' for i in client_manuf]))]
        total_entries['Value Share'] = total_entries['Value Share'].astype(float)
        total_entries = total_entries.nlargest(1,columns="Value Share")    
        comp_lis = list(total_entries[f'{ManufOrTopC}'].str.replace(" Total",'').str.strip())
        dc = (t[t[f'{ManufOrTopC}'].isin(client_manuf) | t[f'{ManufOrTopC}'].isin([i + " Total" for i in client_manuf])]).replace([i + " Total" for i in client_manuf],"Total")
        df = (t[t[f'{ManufOrTopC}'].isin(comp_lis) | t[f'{ManufOrTopC}'].isin([i + " Total" for i in comp_lis])]).replace([i + " Total" for i in comp_lis],"Total")
        
        dc = dc[dc['Net Sales'] > 1000]
        unique_scope = dc[Inscope].unique().tolist()

        df = df[df[Inscope].isin(unique_scope)]

        for subcat in unique_scope:
            if (subcat not in df[Inscope].values) & (subcat!=""):
                df = pd.concat([df, pd.DataFrame({f'{ManufOrTopC}': [df[f'{ManufOrTopC}'].unique()[0]], Inscope: [subcat]})])


        dc = sort_df_excluding_total(dc, Inscope)
        df = sort_df_excluding_total(df, Inscope)

        if not dc.empty:
            client_dictionary[k]=dc
        if not df.empty:
            competitor_dictionary[k]=df

In [74]:
####Sector_Cleaning 
client_manuf_sector_dfs_new = {}
top_competitor_manuf_sector_dfs_new = {}
if len(sectors)!=0:
    split_client_copetitor(manuf_Sector_dfs,client_manuf_sector_dfs_new,top_competitor_manuf_sector_dfs_new,Inscope="Sector")

####Segment_Cleaning
client_manuf_segment_dfs_new = {}
top_competitor_manuf_segment_dfs_new = {}
if len(segments)!=0:
    split_client_copetitor(manuf_Segment_dfs,client_manuf_segment_dfs_new,top_competitor_manuf_segment_dfs_new,Inscope="Segment")

####SubSegment_Cleaning
client_manuf_subsegment_dfs_new = {}
top_competitor_manuf_subsegment_dfs_new = {}
if len(subsegments)!=0:
    split_client_copetitor(manuf_SubSegment_dfs,client_manuf_subsegment_dfs_new,top_competitor_manuf_subsegment_dfs_new,Inscope="SubSegment")

####SubCategory_Cleaning
client_manuf_subcategory_dfs_new = {}
top_competitor_manuf_subcategory_dfs_new = {}
if len(subcategories)!=0:
    split_client_copetitor(manuf_SubCategory_dfs,client_manuf_subcategory_dfs_new,top_competitor_manuf_subcategory_dfs_new,Inscope="SubCategory")

# duplication part 

In [35]:
index = [
    *[0]*((1 if len(client_brands)>1 else 0)+(1 if regions_RET else 0)+(1 if channels_RET else 0)+(1 if market_RET else 0)
          +(1 if regions_CHAN else 0)+(1 if channels_CHAN else 0)+(1 if market_CHAN else 0)
          +(1 if regions_CUST else 0)+(1 if channels_CUST else 0)+(1 if market_CUST else 0)
          +(1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)+(1 if len(client_brands)>1 else 0)),
          
    *[1]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[2]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[3]*((1 if len(client_brands)>1 else 0)+(1 if len(client_brands)>1 else 0)),
    *[4]*((1 if retailerDuplication !=0 else 0)+(1 if channelDuplication !=0 else 0 )+(1 if custDuplication !=0 else 0 )),
    *[5]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[6]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[7]*(1 if len(client_brands)>1 else 0),
    *[8]*( 1 if len(client_brands)>1 else 0),
    *[9]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0))
          ]


# slide 0
duplication_1=[(len(modified_brand_P12M_dfs.keys()) if len(client_brands)>1 else 0)]
duplication_2 = [len(modified_retailer_regions.keys()) if len(regions_RET)>0 else 0, len(modified_retailer_channels) if len(channels_RET)>0 else 0, len(modified_retailer_markets) if len(market_RET)>0 else 0,
                 len(modified_channels_regions.keys()) if len(regions_CHAN)>0 else 0, len(modified_channels_channels) if len(channels_CHAN)>0 else 0, len(modified_channels_markets) if len(market_CHAN) >0 else 0,
                 len(modified_cust_regions.keys()) if len(regions_CUST)>0 else 0, (len(modified_cust_channels) if len(channels_CUST)>0 else 0), (len(modified_cust_markets) if len(market_CUST)>0 else 0)] 

duplication_3 = [(len(modified_sectors_P12M_mix_analysis_dfs.keys()) if len(sectors)>0 else 0), (len(modified_segment_P12M_mix_analysis_dfs.keys()) if len(segments)>0 else 0), (len(modified_subsegment_P12M_mix_analysis_dfs) if len(subsegments)>0 else 0), (len(modified_subcategory_P12M_mix_analysis_dfs) if len(subcategories)>0 else 0)]

duplication_4 = [len(modified_product_sec_seg_P12M_dfs.keys())]

# Slide 1
duplication_5 = [(len(modified_sector_Trade_Margin_dfs.keys()) if len(sectors)!=0 else 0), (len(modified_segment_Trade_Margin_dfs.keys()) if len(segments)!=0 else 0), (len(modified_subsegment_Trade_Margin_dfs) if len(subsegments)!=0 else 0), (len(modified_subcategory_Trade_Margin_dfs) if len(subcategories)!=0 else 0)]

#Slide 2
duplication_6 = [(len(merged_data_sector.keys()) if len(sectors)!=0 else 0), (len(merged_data_segment.keys()) if len(segments)!=0 else 0), (len(merged_data_subsegment) if len(subsegments)!=0 else 0), (len(merged_data_subcategory) if len(subcategories)!=0 else 0)]

# Slide 3
duplication_7 = [len(modified_product_dfs.keys()),len(modified_brand_product_dfs.keys())]
#slide 4
duplication_8 = [retailerDuplication,channelDuplication,custDuplication]

# Slide 5
duplication_9=[(len(secfinaldic.keys())if len(sectors)!=0 else 0),(len(segfinaldic.keys())if len(segments)!=0 else 0),(len(subsegfinaldic.keys())if len(subsegments)!=0 else 0),(len(subcatfinaldic.keys())if len(subcategories)!=0 else 0)]

# Slide 5
duplication_10 = [(len(modified_sector_P12M.keys()) if len(sectors)!=0 else 0), (len(modified_segment_P12M.keys()) if len(segments)!=0 else 0), (len(modified_subsegment_P12M) if len(subsegments)!=0 else 0), (len(modified_subcategory_P12M) if len(subcategories)!=0 else 0)]

# Slide 6
duplication_11 = [len(modified_products_p12m)+ len(modified_products_p12m_brand)]

# Slide 7 
duplication_12 = [len(modified_product_P12M_dfs) + len(modified_brand_product_P12M_dfs)]

# Slide 8
duplication_13 = [(len(client_manuf_sector_dfs_new.keys()) if len(sectors)!=0 else 0), (len(client_manuf_segment_dfs_new.keys()) if len(segments)!=0 else 0), (len(client_manuf_subsegment_dfs_new) if len(subsegments)!=0 else 0), (len(client_manuf_subcategory_dfs_new) if len(subcategories)!=0 else 0)]


duplication = duplication_1 + duplication_2 + duplication_3 + duplication_4 + duplication_5 + duplication_6 + duplication_7 + duplication_8+duplication_9 + duplication_10 + duplication_11+duplication_12+duplication_13

duplication = [item for item in duplication if item !=0]

section_1 = (["Mix Analysis by brand"] if len(client_brands)>1 else [])
section_2 = (["Mix Analysis by Retailer for Region"] if len(regions_RET)!=0 else [])+ (["Mix Analysis by Retailer for Channel"] if len(channels_RET)!=0 else [])+ (["Mix Analysis by Retailer for Market"] if len(market_RET)!=0 else [])+(["Mix Analysis by Channel for Region"] if len(regions_CHAN)!=0 else [])+ (["Mix Analysis by Channel for Channel"] if len(channels_CHAN)!=0 else [])+ (["Mix Analysis by Channel for Market"] if len(market_CHAN)!=0 else [])+([f"Mix Analysis by {customareas} for Region"] if len(regions_CUST)!=0 else [])+ ([f"Mix Analysis by {customareas} for Channel"] if len(channels_CUST)!=0 else [])+ ([f"Mix Analysis by {customareas} for Market"] if len(market_CUST)!=0 else [])
section_3 = (["Mix Analysis by Sector"] if len(sectors)!=0 else [])+ (["Mix Analysis by Segment"] if len(segments)!=0 else [])+ (["Mix Analysis by SubSegment"] if len(subsegments)!=0 else [])+ (["Mix Analysis by SubCategory"] if len(subcategories)!=0 else [])
section_4 = ["Mix Analysis by"+ f'{prodORitem}' if len(client_brands)>1 else []]

section_5 = (["Trade Margin Analysis by Sector"] if len(sectors)!=0 else [])+ (["Trade Margin Analysis by Segment"] if len(segments)!=0 else [])+ (["Trade Margin Analysis by SubSegment"] if len(subsegments)!=0 else [])+(["Trade Margin Analysis by SubCategory"] if len(subcategories)!=0 else [])

section_6 = (["Sector KPI"] if len(sectors)!=0 else [])+ (["Segment KPI"] if len(segments)!=0 else [])+ (["SubSegment KPI"] if len(subsegments)!=0 else [])+ (["SubCategory KPI"] if len(subcategories)!=0 else [])

section_7 = ("SKU KPIs Summary By Manufacture" if len(client_brands)>1 else [],"SKU KPIs Summary By Brand" if len(client_brands)>1 else [])

section_8 = ['Mix Matrix By Retailer'if retailerDuplication != 0 else[] ,'Mix Matrix By Channel'if channelDuplication != 0 else[],'Mix Matrix By Custom Region'if custDuplication != 0 else[]]#,*section_names_slide4]#*section_names_slide3,*section_names_slide4]

section_9=["Mix Matrix By Brands by Sector" if len(sectors)!=0 else [],"Mix Matrix By Brands by Segment" if len(segments)!=0 else [],"Mix Matrix By Brands by SubSegment"if len(subsegments)!=0 else [],"Mix Matrix By Brands by SubCategory"if len(subcategories)!=0 else []]

section_10 = (["Sector Spending Pool"] if len(sectors)!=0 else [])+ (["Segment Spending Pool"] if len(segments)!=0 else [])+ (["SubSegment Spending Pool"] if len(subsegments)!=0 else [])+ (["SubCategory Spending Pool"] if len(subcategories)!=0 else [])

section_11 = (["Product Spending Pool"] if len(client_brands)>1 else [])

section_12 = [("SKU Profitability" if len(client_brands)>1 else [])]

section_13 = (["Trade Margin Table By Sector"] if len(sectors)!=0 else [])+ (["Trade Margin Table By Segment"] if len(segments)!=0 else [])+ (["Trade Margin Table By SubSegment"] if len(subsegments)!=0 else [])+ (["Trade Margin Table By SubCategory"] if len(subcategories)!=0 else [])

section_names = [*section_1 , *section_2 , *section_3 , *section_4 , *section_5 , *section_6 , *section_7 , *section_8, *section_9 , *section_10 , *section_11,*section_12,*section_13]

section_names = [item for item in section_names if item !=[]]
 

path = os.getcwd() + "\Financials base.pptx"
new_pre = os.getcwd() + '\Financials Slide Duplicate.pptx'

In [36]:
print(index)
print(duplication)
print(section_names)
print(len(index))
print(len(duplication))
print(len(section_names))
print(sum(duplication))


[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 5, 5, 5, 6, 6, 6, 7, 8, 9, 9, 9]
[64, 20, 20, 5, 5, 5, 94, 5, 5, 5, 5, 5, 5, 94, 222, 5, 5, 5, 5, 5, 5, 320, 316, 5, 5, 5]
['Mix Analysis by brand', 'Mix Analysis by Retailer for Region', 'Mix Analysis by Retailer for Channel', 'Mix Analysis by Sector', 'Mix Analysis by Segment', 'Mix Analysis by SubSegment', 'Mix Analysis byItem', 'Trade Margin Analysis by Sector', 'Trade Margin Analysis by Segment', 'Trade Margin Analysis by SubSegment', 'Sector KPI', 'Segment KPI', 'SubSegment KPI', 'SKU KPIs Summary By Manufacture', 'SKU KPIs Summary By Brand', 'Mix Matrix By Brands by Sector', 'Mix Matrix By Brands by Segment', 'Mix Matrix By Brands by SubSegment', 'Sector Spending Pool', 'Segment Spending Pool', 'SubSegment Spending Pool', 'Product Spending Pool', 'SKU Profitability', 'Trade Margin Table By Sector', 'Trade Margin Table By Segment', 'Trade Margin Table By SubSegment']
26
26
26
1240


In [37]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [75]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

In [76]:
prs = Presentation(new_pre)

In [77]:
p=0
if len(client_brands)>1:
    Mixanalysis(prs,modified_brand_P12M_dfs,duplication[p],Inscop='Brand',position = calculate_position(p),label_col='Top Brands')
    p+=1
if len(regions_RET)!=0:
    Mixanalysis(prs,modified_retailer_regions,duplication[p],Inscop='Retailer',position = calculate_position(p),label_col='Region')
    p+=1
if len(channels_RET)!=0:
    Mixanalysis(prs,modified_retailer_channels,duplication[p],Inscop='Retailer',position = calculate_position(p),label_col='Channel')
    p+=1
if len(market_RET)!=0:
    Mixanalysis(prs,modified_retailer_markets,duplication[p],Inscop='Retailer',position = calculate_position(p),label_col='Market')
    p+=1

if len(regions_CHAN)!=0:
    Mixanalysis(prs,modified_channels_regions,duplication[p],Inscop='Channel',position = calculate_position(p),label_col='Region')
    p+=1
if len(channels_CHAN)!=0:
    Mixanalysis(prs,modified_channels_channels,duplication[p],Inscop='Channel',position = calculate_position(p),label_col='Channel')
    p+=1
if len(market_CHAN)!=0:
    Mixanalysis(prs,modified_channels_markets,duplication[p],Inscop='Channel',position = calculate_position(p),label_col='Market')
    p+=1

if len(regions_CUST)!=0:
    Mixanalysis(prs,modified_cust_regions,duplication[p],Inscop=f"{customareas}",position = calculate_position(p),label_col='Region')
    p+=1
if len(channels_CUST)!=0:
    Mixanalysis(prs,modified_cust_channels,duplication[p],Inscop=f"{customareas}",position = calculate_position(p),label_col='Channel')
    p+=1
if len(market_CUST)!=0:
    Mixanalysis(prs,modified_cust_markets,duplication[p],Inscop=f"{customareas}",position = calculate_position(p),label_col='Market')
    p+=1


if len(sectors)!=0:
    Mixanalysis(prs,modified_sectors_P12M_mix_analysis_dfs,duplication[p],Inscop='Sector',position = calculate_position(p),label_col='Sector')
    p+=1
if len(segments)!=0:         
    Mixanalysis(prs,modified_segment_P12M_mix_analysis_dfs,duplication[p],Inscop="Segment",position = calculate_position(p),label_col="Segment")
    p+=1
if len(subsegments)!=0:
    Mixanalysis(prs,modified_subsegment_P12M_mix_analysis_dfs,duplication[p],Inscop="SubSegment",position = calculate_position(p),label_col="SubSegment")
    p+=1
if len(subcategories)!=0:
    Mixanalysis(prs,modified_subcategory_P12M_mix_analysis_dfs,duplication[p],Inscop="SubCategory",position = calculate_position(p),label_col="SubCategory")
    p+=1
if len(client_brands)>1:
    Mixanalysis(prs,modified_product_sec_seg_P12M_dfs,duplication[p],Inscop=f'{prodORitem}',position = calculate_position(p),label_col=f'{prodORitem}')
    p+=1



In [78]:
if len(sectors)!=0:
    TradeMargin(prs,modified_sector_Trade_Margin_dfs,duplication[p],position = calculate_position(p),InScope='Sector')
    p+=1
if len(segments)!=0:
    TradeMargin(prs,modified_segment_Trade_Margin_dfs,duplication[p],position = calculate_position(p),InScope="Segment")
    p+=1
if len(subsegments)!=0:
    TradeMargin(prs,modified_subsegment_Trade_Margin_dfs,duplication[p],position = calculate_position(p),InScope="SubSegment")
    p+=1
if len(subcategories)!=0:
    TradeMargin(prs,modified_subcategory_Trade_Margin_dfs,duplication[p],position = calculate_position(p),InScope="SubCategory")
    p+=1        

In [79]:
if len(sectors)!=0:
    sectorKPI(prs, merged_data_sector, duplication[p], position=calculate_position(p),Scope= "Sector")
    p+=1
if len(segments)!=0:
    sectorKPI(prs, merged_data_segment, duplication[p], position=calculate_position(p),Scope= "Segment")
    p+=1

if len(subsegments)!=0:
    sectorKPI(prs, merged_data_subsegment, duplication[p], position=calculate_position(p),Scope= "SubSegment")
    p+=1

if len(subcategories)!=0:
    sectorKPI(prs, merged_data_subcategory, duplication[p], position=calculate_position(p),Scope= "SubCategory") 
    p+=1

In [80]:
if len(client_brands)>1:

    SkuKpis(prs,modified_product_dfs,duplication[p],position = calculate_position(p))
    p+=1
    SkuKpis(prs,modified_brand_product_dfs,duplication[p],position = calculate_position(p))
    p+=1

In [81]:
# mixMatrix(prs,secretailerTotal,position = 0)
pos = calculate_position(p)
if retailerDuplication!=0:
    for key,val in retailerDic.items():
        if val:
            mixMatrix(prs,val,position = pos)
            print(pos)

            pos +=len(retailerDic[key].keys())

    p+=1        
if channelDuplication!=0:
    for key,val in channelDic.items():
        if val:
            mixMatrix(prs,val,position = pos)
            pos +=len(channelDic[key].keys())
    p+=1       
if custDuplication!=0:
    for key,val in custDic.items():
        if val:
            mixMatrix(prs,val,position = pos)
            pos +=len(custDic[key].keys())
    p+=1        


In [82]:
pos = calculate_position(p)
if len(sectors)!=0:
    mixMatrix(prs,secfinaldic,position = calculate_position(p),slideby="bybrand")
    p+=1
if len(segments)!=0:
    mixMatrix(prs,segfinaldic,position = calculate_position(p),slideby="bybrand")
    p+=1
if len(subsegments)!=0:
    mixMatrix(prs,subsegfinaldic,position = calculate_position(p),slideby="bybrand")
    p+=1
if len(subcategories)!=0:
    mixMatrix(prs,subcatfinaldic,position = calculate_position(p),slideby="bybrand")
    p+=1

In [84]:
if len(sectors)!=0:
    sectorSpendingPool(prs, modified_sector_P12M,duplication[p], position =calculate_position(p),Scope= "Sector")
    p+=1

if len(segments)!=0:
    sectorSpendingPool(prs, modified_segment_P12M,duplication[p], position =calculate_position(p),Scope="Segment")
    p+=1

if len(subsegments)!=0:
    sectorSpendingPool(prs, modified_subsegment_P12M,duplication[p], position =calculate_position(p),Scope="SubSegment")
    p+=1

if len(subcategories)!=0:
    sectorSpendingPool(prs, modified_subcategory_P12M,duplication[p], position =calculate_position(p),Scope= "SubCategory")    
    p+=1

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [85]:
if len(client_brands)>1:

    combinedmanufbranddfpool=modified_products_p12m
    combinedmanufbranddfpool.update(modified_products_p12m_brand)
    productSpendingPool(prs,combinedmanufbranddfpool, duplication[p], position= calculate_position(p))
    p+=1

In [86]:
if len(client_brands)>1:
    combinedmanufbranddf=modified_product_P12M_dfs
    combinedmanufbranddf.update(modified_brand_product_P12M_dfs)
    SKUProfitability(prs,combinedmanufbranddf,duplication[p],position = calculate_position(p))
    p+=1

In [87]:
if len(sectors)!=0:
    TradeMarginTable(prs,client_manuf_sector_dfs_new,top_competitor_manuf_sector_dfs_new,duplication[p],position = calculate_position(p),Inscope="Sector")
    p+=1
if len(segments)!=0:  
    TradeMarginTable(prs,client_manuf_segment_dfs_new,top_competitor_manuf_segment_dfs_new,duplication[p],position = calculate_position(p),Inscope="Segment")
    p+=1
if len(subsegments)!=0:  
    TradeMarginTable(prs,client_manuf_subsegment_dfs_new,top_competitor_manuf_subsegment_dfs_new,duplication[p],position = calculate_position(p),Inscope="SubSegment")
    p+=1   
if len(subcategories)!=0:  
    TradeMarginTable(prs,client_manuf_subcategory_dfs_new,top_competitor_manuf_subcategory_dfs_new,duplication[p],position = calculate_position(p),Inscope="SubCategory")
    p+=1


In [88]:
outputPath=os.getcwd() +"\\FinalOutput"+f"\\Financials_Output_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)
# app = win32.Dispatch("PowerPoint.Application")/
# presentation = app.Presentations.Open(outputPath)

In [51]:
final=os.getcwd() +"\\FinalOutput"+f"\\Financials_Output_{datetime.today().strftime("%d-%m")}.pptx"

In [52]:
# Example usage
pptx_path = outputPath  # Replace with the actual path to your PPTX file
output_pptx_path=final
open_chart_data_in_excel(pptx_path,output_pptx_path)


Slide 1: Opened Excel workbook: Book1
Slide 2: Opened Excel workbook: Book1
Slide 3: Opened Excel workbook: Book1
Slide 4: Opened Excel workbook: Book1
Slide 5: Opened Excel workbook: Book1
Slide 6: Opened Excel workbook: Book1
Slide 7: Opened Excel workbook: Book1
Slide 8: Opened Excel workbook: Book1
Slide 9: Opened Excel workbook: Book1
Slide 10: Opened Excel workbook: Book1
Slide 11: Opened Excel workbook: Book1
Slide 12: Opened Excel workbook: Book1
Slide 13: Opened Excel workbook: Book1
Slide 14: Opened Excel workbook: Book1
Slide 15: Opened Excel workbook: Book1
Slide 16: Opened Excel workbook: Book1
Slide 17: Opened Excel workbook: Book1
Slide 18: Opened Excel workbook: Book1
Slide 19: Opened Excel workbook: Book1
Slide 20: Opened Excel workbook: Book1
Slide 21: Opened Excel workbook: Book1
Slide 22: Opened Excel workbook: Book1
Slide 23: Opened Excel workbook: Book1
Slide 24: Opened Excel workbook: Book1
Slide 25: Opened Excel workbook: Book1
Slide 26: Opened Excel workbook: B